New notebook to test the difference between the tech rec mongo db and gal alum.
Currently, code working for gal alum but not tech recruiters.
1. Copy progress from mongodb_test
2. Replicate the gal alum process from
* Connection to mongo db
* Pull data into df
* Manipulate to correct format
* Create csv of all companies
* Create csvs of individual companies

3. New notebook section for tech recruiters

In [1]:
import pandas as pd
import mongo
import ast
import os

# Galvanize Alumni CSV build

In [3]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'gal_alum')
cursor = mongo.coll.find({}, {'_id':0})
df = pd.DataFrame(cursor)
df = df.T

In [4]:
index = list()
results = list()

for idx, row in df.iterrows():
    index.append(idx)
    results.append(row[row.notnull()].values[0])

clean_df = pd.DataFrame(results, index=index)
clean_df.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df.head()

,Scrape Results
University of Texas at Austin,People Link 404
Apple,{'Erin Gong': 'https://www.linkedin.com/in/eri...
Dell Technologies,{'Daniel Donovan': 'https://www.linkedin.com/i...
IBM,{'James Helfrich': 'https://www.linkedin.com/i...
National Instruments,No results


# Austin Technical Recruiters CSV build
## Load data from mongo

In [5]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'ATX_tech_rec')
cursor = mongo.coll.find({}, {'_id':0})

df_tr = pd.DataFrame(cursor)
df_tr = df_tr.T

## Austin Tech Recruiters pipeline error corrections
My dataframe pipeline is failing for the tech recruiters mongo pull. Successively passing the Alumni pull with the same process. Steps to troubleshoot proceed.

### Issue: Pandas takes the assumption of the data series for the DataFrame based upon the __first item__ in the list/array being built. The first item in the list/array was a string. 

### Fix: Force-inserting an item into the list remedied this. Line 9 & 10 below.

In [6]:
# pipeline for ATX Tech Recruiters
index_tr = list()
results_tr = list()

for idx, row in df_tr.iterrows():
    index_tr.append(idx)
    results_tr.append(row[row.notnull()].values[0])

results_tr.insert(0, 'ignore')
index_tr.insert(0, 'ignore')    
clean_df_tr = pd.DataFrame(results_tr, index=index_tr)
clean_df_tr.drop('ignore', axis=0, inplace=True)
clean_df_tr.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df_tr.head()

,Scrape Results
9Gauge Partners,{'Carrie Guillory': 'https://www.linkedin.com/...
Abbott,{'Brandy Williams Broden': 'https://www.linked...
Aceable,"{'Andrew Bolisay (he, him, his)': 'https://www..."
AdbaKx,Company Page 404
Addison Group,{'Hattie Cox': 'https://www.linkedin.com/in/ha...


# Convert DataFrame to CSVs for each company (observations)

## Create local directory for csv
### Variables should be switched between `tech_rec` and `alumni` for saving between the two types of variables.

In [34]:
''' change below assignment for saving different CSVs'''
contacts = clean_df_tr['Scrape Results'] #clean_df
employee_type = 'tech_rec' #alumni

In [43]:
'''Make Directories'''
# os.mkdir('../data/alumni')
os.mkdir('../data/tech_rec')

In [45]:
# dict_flag = contacts.apply(lambda x: isinstance(x, dict))
# contacts['Scrape Results'] = contacts['Scrape Results'].apply(lambda x: ast.literal_eval(x))
contacts = contacts[dict_flag] #clean_df
employee_type = 'tech_rec' #alumni

for i, c in zip(contacts.index, contacts):
    co_contacts = pd.DataFrame(c.values(), index=c.keys())
    co_contacts.to_csv('../data/{0}/{1}.csv'.format(employee_type, i))

# gDrive interaction testing

In [111]:
import gspread